In [1]:
import os
import pickle
import numpy as np
from queries import Simbad
from astropy.io import fits
import astropy.units as u
from astropy.coordinates import SkyCoord
from IPython.display import clear_output

import sys
if sys.platform=='darwin':
    top_path='/Users/codydirks/PGCC/'
else:
    top_path='/DataDisk/datafiles/PGCC_HST/'

x1d_dir=top_path+'x1d_files/'

def find_in_tgas(key,field):
    filenames=[top_path+'tgas_data/TgasSource_000-000-0'+'{:02}'.format(i)+'.fits' for i in range(16)]
    for filename in filenames:
        gaia_data=fits.open(filename)[1].data
        #gaia_entry=gaia_data[gaia_data[field]==key]
        gaia_entry=[i for i,item in enumerate(gaia_data[field]==key) if item==True]
        if len(gaia_entry)>0:
            return filenames.index(filename),gaia_entry[0]
    return None
        
#Returns angular separation between two Ra/Dec coordinates in degrees
def get_ang_sep(ra1,dec1,ra2,dec2):
    ra1_rad=ra1*np.pi/180.
    dec1_rad=dec1*np.pi/180.
    ra2_rad=ra2*np.pi/180.
    dec2_rad=dec2*np.pi/180.

    del_ra=ra2_rad-ra1_rad
    del_ras=np.sin(del_ra)
    del_rac=np.cos(del_ra)
    dec1s=np.sin(dec1_rad)
    dec2s=np.sin(dec2_rad)
    dec1c=np.cos(dec1_rad)
    dec2c=np.cos(dec2_rad)
    result=(180./np.pi)*np.arctan(np.sqrt((dec2c**2)*(del_ras**2)+(dec1c*dec2s-dec1s*dec2c*del_rac)**2)/(dec1s*dec2s+dec1c*dec2c*del_rac))
    if result < 0:
        return result+180.
    else:
        return result
    
    
    
def store_results(results):
    filename='sightline_pgcc_gaia_results.txt'
    with open(filename,'w') as myfile:
        for entry in results:
            myfile.write('|'.join([entry[0],
                                   str(entry[1][0].value)+' '+str(entry[1][1].value),
                                   str(entry[2]),
                                   str(entry[3]),
                                   '\n']))

def get_r_dist(pgcc,sightline_ra,sightline_dec):
    angle=pgcc['gau_position_angle']
    sin=np.sin(-angle)
    cos=np.cos(-angle)
    a=pgcc['gau_major_axis']/2.
    b=pgcc['gau_minor_axis']/2.
    gal=SkyCoord(ra=sightline_ra*u.degree,dec=sightline_dec*u.degree,frame='icrs').galactic
    del_l=60.*(gal.l.value-pgcc['glon'])
    del_b=60.*(gal.b.value-pgcc['glat'])
    return np.sqrt((((cos*del_l+sin*del_b)/a)**2+((sin*del_l-cos*del_b)/b)**2))

In [4]:
x1d_path=top_path+'x1d_files/'
sightlines=[x for x in os.listdir(x1d_path) 
            if os.path.isdir(x1d_path+x+'/E140H')==True and x.startswith('.')==False]
pgcc_hdu=fits.open(top_path+'HFI_PCCS_GCC_R2.02.fits')
pgcc_data=pgcc_hdu[1].data
pgcc_hdu.close()
#pgcc_data=pgcc_data[pgcc_data]
sightlines.sort(key=lambda x: (x[0],int(x.strip('HDECPD-'))))
print(sightlines)

['CPD-592603', 'HD1383', 'HD4768', 'HD10250', 'HD13841', 'HD14818', 'HD15137', 'HD21551', 'HD23180', 'HD23478', 'HD23873', 'HD24398', 'HD24534', 'HD25443', 'HD32630', 'HD36285', 'HD36959', 'HD37021', 'HD37061', 'HD37367', 'HD37903', 'HD40893', 'HD040893', 'HD42111', 'HD43582', 'HD44173', 'HD46300', 'HD48915', 'HD50138', 'HD51285', 'HD52266', 'HD52329', 'HD62542', 'HD69106', 'HD72089', 'HD72350', 'HD72754', 'HD75309', 'HD93205', 'HD93206', 'HD93222', 'HD93237', 'HD94414', 'HD94493', 'HD99857', 'HD100546', 'HD101131', 'HD101436', 'HD104705', 'HD106111', 'HD106343', 'HD106943', 'HD108002', 'HD108610', 'HD108639', 'HD108927', 'HD110434', 'HD111934', 'HD112999', 'HD114886', 'HD115071', 'HD115455', 'HD116781', 'HD117111', 'HD122879', 'HD124314', 'HD124979', 'HD137595', 'HD138679', 'HD147196', 'HD147683', 'HD148594', 'HD148937', 'HD152249', 'HD152424', 'HD152590', 'HD157857', 'HD165918', 'HD173297', 'HD175640', 'HD177989', 'HD185418', 'HD186688', 'HD191877', 'HD192577', 'HD202347', 'HD203374'

In [5]:
sightline_pgcc_gaia_data=[]

for sightline in sightlines:
    #clear_output()
    #print sightlines.index(sightline)+1, '/', len(sightlines)
    print('{:>12}'.format(sightline),end='   ')
    coords=Simbad.Position(sightline)
    nearest=9999
    nearest_pgcc=[]
    for i in range(len(pgcc_data)):
        pgcc_ra,pgcc_dec=pgcc_data[i][3:5]
        if abs(coords[0].value-pgcc_ra)<10 and abs(coords[1].value-pgcc_dec)<10:
            angsep=get_ang_sep(coords[0].value,coords[1].value,pgcc_ra,pgcc_dec)
            if angsep<nearest:
                nearest=angsep
                nearest_pgcc=i
    
    print('PGCC #','{:>5}'.format(nearest_pgcc), 
          '{:>.1f}'.format(round(get_r_dist(pgcc_data[nearest_pgcc],coords[0].value,coords[1].value),1)), end='    ')
    
    idlist=Simbad.IDList(sightline)
    hip=[x for x in idlist if x.startswith('HIP')]
    tyc=[x for x in idlist if x.startswith('TYC')]
    if len(hip)>0 or len(tyc)>0:
        if len(hip)>0:
            hip_id=int(hip[0].split()[1])
            gaia_entry=find_in_tgas(hip_id,'hip')
        elif len(tyc)>0:
            tyc_id=tyc[0].split()[1]
            gaia_entry=find_in_tgas(tyc_id,'tycho2_id')
    print('Gaia #:',gaia_entry)
    sightline_pgcc_gaia_data.append([sightline,coords,nearest_pgcc,gaia_entry])


  CPD-592603   PGCC #  6916 2.1    Gaia #: (10, 105591)
      HD1383   

/Users/codydirks/anaconda3/envs/astroconda/lib/python3.5/site-packages/ipykernel_launcher.py:69: RuntimeWarning: divide by zero encountered in double_scalars


PGCC #  5473 inf    Gaia #: (0, 119216)
      HD4768   PGCC #  9198 0.8    Gaia #: (0, 116833)
     HD10250   PGCC # 10571 7.6    Gaia #: None
     HD13841   PGCC #  1720 3.6    Gaia #: (0, 133397)
     HD14818   PGCC #  3783 9.0    Gaia #: (0, 132557)
     HD15137   PGCC #   122 5.4    Gaia #: (0, 128920)
     HD21551   PGCC #  5681 4.2    Gaia #: (0, 53337)
     HD23180   PGCC #   172 6.4    Gaia #: None
     HD23478   PGCC #  1195 6.1    Gaia #: (0, 41868)
     HD23873   PGCC #  4402 8.1    Gaia #: (0, 8843)
     HD24398   PGCC #  3534 9.1    Gaia #: None
     HD24534   PGCC #  8105 3.1    Gaia #: (0, 26057)
     HD25443   PGCC #  9605 6.4    Gaia #: (1, 6763)
     HD32630   PGCC #  7208 7.7    Gaia #: None
     HD36285   PGCC #  9945 inf    Gaia #: None
     HD36959   PGCC #  1934 1.1    Gaia #: None
     HD37021   PGCC #   645 inf    Gaia #: None
     HD37061   PGCC #    91 4.4    Gaia #: None
     HD37367   PGCC #  2243 7.8    Gaia #: None
     HD37903   PGCC #   240 1.8    Gaia 

In [6]:
#store_results(sightline_pgcc_gaia_data)

In [68]:
import healpy as hp
m=hp.fitsfunc.read_map('/Users/codydirks/PGCC/CO/HFI_CompMap_CO-Type3_2048_R1.10.fits')

NSIDE = 2048
ORDERING = NESTED in fits file
INDXSCHM = IMPLICIT


/Users/codydirks/anaconda3/envs/astroconda/lib/python3.5/site-packages/healpy/fitsfunc.py:326: UserWarning: No INDXSCHM keyword in header file : assume IMPLICIT
  "assume {}".format(schm))


Ordering converted to RING


In [121]:
import csv
import thesis_tools
with open('primary_sightlines.csv','r') as csvfile:
    inp=csv.reader(csvfile)
    primary_sightlines=next(inp)

sl_data=thesis_tools.load_data(primary_sightlines)
all_data=thesis_tools.load_results(sl_data)

/Users/codydirks/python/thesis_tools/loaders.py:110: RuntimeWarning: divide by zero encountered in log10
  n_dict[ion]=round(np.log10(n_tot),3)
/Users/codydirks/anaconda3/envs/astroconda/lib/python3.5/site-packages/numpy/core/fromnumeric.py:2909: RuntimeWarning: Mean of empty slice.
  out=out, **kwargs)
/Users/codydirks/anaconda3/envs/astroconda/lib/python3.5/site-packages/numpy/core/_methods.py:80: RuntimeWarning: invalid value encountered in double_scalars
  ret = ret.dtype.type(ret / rcount)
/Users/codydirks/anaconda3/envs/astroconda/lib/python3.5/site-packages/numpy/core/_methods.py:135: RuntimeWarning: Degrees of freedom <= 0 for slice
  keepdims=keepdims)
/Users/codydirks/anaconda3/envs/astroconda/lib/python3.5/site-packages/numpy/core/_methods.py:105: RuntimeWarning: invalid value encountered in true_divide
  arrmean, rcount, out=arrmean, casting='unsafe', subok=False)
/Users/codydirks/anaconda3/envs/astroconda/lib/python3.5/site-packages/numpy/core/_methods.py:127: RuntimeWarni

In [124]:
import matplotlib.pyplot as plt
from matplotlib.backends.backend_pdf import PdfPages
import seaborn as sb
blue=sb.xkcd_palette(['windows blue'])[0]
amber=sb.xkcd_palette(['amber'])[0]
red=sb.xkcd_palette(['pale red'])[0]
%matplotlib inline

with PdfPages('multipage.pdf') as pdf:
    for sl,data in all_data.groupby('Sightline'):
        fig=plt.figure(1,figsize=(16,12),facecolor='w')
        row=data.sort_values('CO',ascending=False).iloc[0]
        if row['CO']>15:
            col=blue
        else:
            col=red
        entry=[i for i in sightline_pgcc_gaia_data if i[0]==sl][0]
        ra,dec=entry[1]
        gal=SkyCoord(ra=ra,dec=dec,frame='icrs').galactic
        l=gal.l.value
        b=gal.b.value
        if l>180:
            l=l-360
        hp.cartview(m,fig=1,lonra=[l-1,l+1],latra=[b-1,b+1],min=0,max=75,cbar=False,margins=[0.01,0.01,0.01,0.01],title='')
        hp.projplot(l, b,lonlat=True,marker='*',color=col,mec='w',mew=2,markersize=30)
        hp.graticule(dpar=0.5)
        fig.suptitle(sl,fontsize=40,backgroundcolor='w')
        pdf.savefig()
        plt.close()

89.65518617784801 95.30924269438557 -62.673527156193515 -57.01416053310374
The interval between parallels is 0 deg 30.00'.
The interval between meridians is 0 deg 30.00'.
102.58167871440719 108.13824588394289 -60.96176948683767 -55.19937378817493
The interval between parallels is 0 deg 30.00'.
The interval between meridians is 0 deg 30.00'.
84.99678271475378 90.65145372054036 -58.654723470165 -52.99597243093159
The interval between parallels is 0 deg 30.00'.
The interval between meridians is 0 deg 30.00'.
87.59662980759148 93.25326263412444 -50.16264384274345 -44.50585543375351
The interval between parallels is 0 deg 30.00'.
The interval between meridians is 0 deg 30.00'.
91.10877527450218 96.75882382660987 131.54664940131036 137.2100410053686
The interval between parallels is 0 deg 30.00'.
The interval between meridians is 0 deg 30.00'.
77.10461302974475 82.71774484668745 -17.99403429962351 -12.292745604464937
The interval between parallels is 0 deg 30.00'.
The interval between meridi